In [1]:
# Colab 사용 시 드라이브 연결

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# 라이브러리 설치
!python3 -m pip install -U demucs
!pip install ffmpeg-python
!pip3 install transkun
!pip install mido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 6.8 MB/s eta 0:00:00
  Created wheel for demucs: filename=demucs-4.0.1-py3-none-any.w

In [2]:
# Music Seperate(음원 분리)
import demucs.separate
import shlex
demucs.separate.main(shlex.split('--mp3 -n htdemucs_6s "/content/drive/MyDrive/Music_SDB/엔플라잉 - 옥탑방 가사 [ ezmp3.cc ].mp3"'))

Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/5c90dfd2-34c22ccb.th" to /root/.cache/torch/hub/checkpoints/5c90dfd2-34c22ccb.th
100%|██████████| 52.4M/52.4M [00:00<00:00, 65.3MB/s]


Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/separated/htdemucs_6s
Separating track /content/drive/MyDrive/Music_SDB/엔플라잉 - 옥탑방 가사 [ ezmp3.cc ].mp3


100%|██████████████████████████████████████████████████████████████████████| 216.45/216.45 [00:13<00:00, 15.57seconds/s]


In [1]:
# Music Transcription(피치 추적)

import os
import glob
import mido

# MIDI 파일 저장 경로
midi_folder = '/content/midi/'

# 음악 파일 경로(음원 분리 된 mp3파일이 있는 경로)
music_folder = '/content/separated/htdemucs_6s/'

# 폴더 내 모든 mp3 파일 찾기
mp3_files = glob.glob(os.path.join(music_folder, '**/*.mp3'), recursive=True)

# 각 악기에 맞는 MIDI 프로그램 번호 설정 (General MIDI 기준)
instrument_programs = {
    'piano': 0,    # 0: Acoustic Grand Piano
    'guitar': 24,  # 24: Acoustic Guitar (nylon)
    'bass': 32     # 32: Acoustic Bass
}

# MIDI 파일에서 악기 변경 함수
def change_instrument(midi_file_path, program_number):
    # MIDI 파일 열기
    mid = mido.MidiFile(midi_file_path)

    # 각 트랙의 메시지 수정
    for i, track in enumerate(mid.tracks):
        for msg in track:
            if msg.type == 'program_change':
                # 기존 프로그램 번호를 새로운 프로그램 번호로 변경
                msg.program = program_number

    # 수정된 MIDI 파일 저장
    mid.save(midi_file_path)

# 각 mp3 파일마다 transkun 명령어 실행 전 파일 이름 확인
for mp3_file in mp3_files:
    file_name = os.path.splitext(os.path.basename(mp3_file))[0]
    midi_file_path = os.path.join(midi_folder, file_name + ".midi")

    # 기본 악기는 피아노로 설정
    instrument = 'piano'
    program_number = instrument_programs[instrument]

    # 파일 이름에 'piano', 'guitar', 'bass'가 포함되어 있는지 확인하여 악기 설정
    if 'piano' in mp3_file.lower():
        instrument = 'piano'
        program_number = instrument_programs[instrument]
    elif 'guitar' in mp3_file.lower():
        instrument = 'guitar'
        program_number = instrument_programs[instrument]
    elif 'bass' in mp3_file.lower():
        instrument = 'bass'
        program_number = instrument_programs[instrument]
    else:
        # 해당 악기가 없으면 변환 건너뜀
        print(f'Skipping {mp3_file} (no matching instrument)')
        continue

    # transkun 명령어로 MIDI 파일 생성
    command = f'transkun "{mp3_file}" "{midi_file_path}" --device cuda'
    print(f"Executing: {command}")
    os.system(command)

    # transkun으로 변환된 파일이 존재하는지 확인
    if not os.path.exists(midi_file_path):
        print(f'Failed to generate MIDI file for {mp3_file}')
        continue

    # transkun으로 생성된 MIDI 파일에서 악기 번호 변환
    change_instrument(midi_file_path, program_number)
    print(f'Instrument changed to {instrument} for {midi_file_path}')


Executing: transkun "/content/separated/htdemucs_6s/엔플라잉 - 옥탑방 가사 [ ezmp3.cc ]/bass.mp3" "/content/midi/bass.midi" --device cuda
Instrument changed to bass for /content/midi/bass.midi
Skipping /content/separated/htdemucs_6s/엔플라잉 - 옥탑방 가사 [ ezmp3.cc ]/vocals.mp3 (no matching instrument)
Skipping /content/separated/htdemucs_6s/엔플라잉 - 옥탑방 가사 [ ezmp3.cc ]/other.mp3 (no matching instrument)
Skipping /content/separated/htdemucs_6s/엔플라잉 - 옥탑방 가사 [ ezmp3.cc ]/drums.mp3 (no matching instrument)
Executing: transkun "/content/separated/htdemucs_6s/엔플라잉 - 옥탑방 가사 [ ezmp3.cc ]/piano.mp3" "/content/midi/piano.midi" --device cuda
Instrument changed to piano for /content/midi/piano.midi
Executing: transkun "/content/separated/htdemucs_6s/엔플라잉 - 옥탑방 가사 [ ezmp3.cc ]/guitar.mp3" "/content/midi/guitar.midi" --device cuda
Instrument changed to guitar for /content/midi/guitar.midi
